In [ ]:
import pandas as pd
# Загрузка датасета
file_path_ml = r"modified_data_encode_sorted_datatime.csv"  # Укажите путь к вашему файлу
data_ml = pd.read_csv(file_path_ml)# Сортировка данных по 'client_id' datetime

print(data_ml)

In [10]:
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [18]:
# Создание последовательностей транзакций для каждого клиента
sequence_length = 10  # Используем последние 10 транзакций для каждого клиента

# Группируем по клиенту и создаем последовательности транзакций
sequences = []


for client_id, group in data_ml.groupby('client_id'):
    group = group.sort_values('datetime')  # Сортируем по времени транзакций
    # Преобразуем транзакции в последовательности
    for i in range(len(group) - sequence_length):
        seq = group.iloc[i:i+sequence_length]  # Последовательность из 10 транзакций
        sequences.append(seq[['device_id', 'device_type', 'tran_code', 'mcc', 'card_type', 'pin_inc_count',
                               'card_status', 'sum', 'oper_type', 'balance', 'ip_city', 'day_of_week',
                               'hour_of_day', 'month']].values)

sequences = np.array(sequences)

# Проверим размеры данных
print(f'Размер последовательностей: {sequences.shape}')

Размер последовательностей: (43135, 10, 14)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Размерность входных данных (последовательность из N транзакций)
input_dim = sequences.shape[2]  # Количество признаков на одну транзакцию
encoding_dim = 128  # Размерность скрытого слоя

# Строим автоэнкодер
input_layer = Input(shape=(sequences.shape[1], input_dim))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)

# Компиляция модели
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Обучение автоэнкодера
autoencoder.fit(sequences, sequences, epochs=20, batch_size=32, validation_split=0.2)

# Получаем восстановленные данные
reconstructed = autoencoder.predict(sequences)

# Вычисляем ошибку восстановления для каждой транзакции
reconstruction_error = np.mean(np.abs(sequences - reconstructed), axis=1)

# Определяем порог ошибки для классификации аномалий
threshold = np.percentile(reconstruction_error, 95)  # Например, 95-й перцентиль

# Классификация аномалий
anomalies = reconstruction_error > threshold

# Смотрим на результаты
print(f'Количество аномальных транзакций: {np.sum(anomalies)}')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
# Параметры модели
sequence_length = 10  # Например, 10 транзакций в окне
num_features = 14  # Количество признаков на транзакцию
latent_dim = 64  # Размер латентного пространства

# Создание модели
model = Sequential()

# LSTM слой для обработки временных зависимостей
model.add(LSTM(128, return_sequences=True, input_shape=(sequence_length, num_features)))
model.add(Dropout(0.2))

# Сжимаем информацию в латентное пространство
model.add(LSTM(latent_dim, return_sequences=False))

# Полносвязный слой для восстановления исходных данных
model.add(Dense(128, activation='relu'))
model.add(Dense(num_features, activation='sigmoid'))  # Восстановление данных

# Выходной слой для классификации аномалий
model.add(Dense(1, activation='sigmoid'))  # 1 для бинарной классификации

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(sequences, sequences, epochs=10, batch_size=32)

In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Нормализуем числовые признаки
scaler = MinMaxScaler(feature_range=(0, 1))
numeric_features = ['sum', 'balance', 'pin_inc_count']  # Числовые признаки
data_ml[numeric_features] = scaler.fit_transform(data_ml[numeric_features])

# Выделим последовательности для каждого клиента
sequence_length = 5  # Используем 5 последних транзакций для предсказания следующей

def create_sequences(data_ml, sequence_length=sequence_length):
    sequences = []
    labels = []
    
    for client_id, group in data_ml.groupby('client_id'):
        for i in range(len(group) - sequence_length):
            # Выбираем последовательность из 5 транзакций
            seq = group.iloc[i:i+sequence_length][numeric_features].values
            sequences.append(seq)
            # Признак: следующая транзакция будет нормальной или аномальной (пока без меток)
            labels.append(group.iloc[i+sequence_length]['sum'])  # Можно использовать разные метки для аномалии
            
    return np.array(sequences), np.array(labels)

X, y = create_sequences(data_ml)

# Разделим на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()

# Добавляем слой LSTM
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dropout(0.2))  # Dropout для предотвращения переобучения

# Добавляем полносвязный слой
model.add(Dense(32, activation='relu'))

# Выходной слой
#model.add(Dense(1, activation='linear'))  # Для задачи регрессии (предсказание суммы)
# Или для бинарной классификации:
model.add(Dense(1, activation='sigmoid'))

# Компиляция модели
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Для задачи регрессии
# Или для классификации:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20


E:\anaconda\envs\rinhakii\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1203/1203 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.0104 - mae: 0.0566 - val_loss: 2.3696e-04 - val_mae: 0.0134
Epoch 2/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 3.2273e-04 - mae: 0.0142 - val_loss: 7.7683e-05 - val_mae: 0.0071
Epoch 3/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 2.2570e-04 - mae: 0.0118 - val_loss: 5.6696e-05 - val_mae: 0.0062
Epoch 4/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.9027e-04 - mae: 0.0108 - val_loss: 3.2414e-05 - val_mae: 0.0048
Epoch 5/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.5484e-04 - mae: 0.0097 - val_loss: 7.4024e-05 - val_mae: 0.0068
Epoch 6/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.2576e-04 - mae: 0.0087 - val_loss: 1.7440e-04 - val_mae: 0.0110
Epoch 7/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0140e-04 - mae: 0.0078 - val_loss: 1.4714e-04 - val_mae: 0.0106
Epoch 8/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 8.6791e-05 - mae: 0.0072 - val_loss: 1.4143e-04 - val_mae

In [ ]:
# Оценка на тестовых данных
loss, mae = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, MAE: {mae}")

# Для бинарной классификации
from sklearn.metrics import classification_report, confusion_matrix

y_pred = (model.predict(X_test) > 0.5).astype(int)  # Предсказания: 0 - нормальная, 1 - аномальная

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


In [ ]:
#ВСЕ ЧТО ВЫШЕ - ТСЕТОВЫЕ ВАРИАНТЫ

In [ ]:
#ВСЕ ЧТО ВЫШЕ - ТСЕТОВЫЕ ВАРИАНТЫ

In [ ]:
#ВСЕ ЧТО ВЫШЕ - ТСЕТОВЫЕ ВАРИАНТЫ

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Загружаем данные
df = pd.read_csv("modified_data_encode_sorted_datatime.csv")



# Удаляем ненужные столбцы (например, идентификаторы или текстовые поля)
df = df.drop(columns=['transaction_id', 'expiration_date', 'datetime', 'ip'])


# Получаем список уникальных клиентов
unique_clients = df['client_id'].unique()

# Функция для создания автоэнкодера
def create_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    # Кодировщик
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(32, activation='relu')(encoded)
    encoded = Dense(16, activation='relu')(encoded)
    
    # Декодировщик
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    
    # Создаем модель автоэнкодера
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder


# Счетчик клиентов
client_counter = 1
# Цикл по каждому клиенту
for client_id in unique_clients:
    print(f"Обработка данных для клиента {client_id}...")
    
    # Фильтруем данные для конкретного клиента
    client_data = df[df['client_id'] == client_id]
    
    # Удаляем client_id из данных (он больше не нужен как признак)
    client_data = client_data.drop(columns=['client_id'])
    
    # Масштабируем данные
    scaler = StandardScaler()
    client_data_scaled = scaler.fit_transform(client_data)
    
    # Создаем автоэнкодер
    input_dim = client_data_scaled.shape[1]
    autoencoder = create_autoencoder(input_dim)
    
    # Обучаем автоэнкодер на всех данных клиента (без разделения на обучающую и тестовую выборки)
    autoencoder.fit(
        client_data_scaled, client_data_scaled,  # Обучение без меток
        epochs=50,
        batch_size=32,
        shuffle=True,
        verbose=1
    )
    
    # Сохраняем модель для клиента
    autoencoder.save(f'autoencoder_client_{client_id}.h5')
    print(f"Модель для клиента {client_id} сохранена.№ {client_counter}\n")
    # Увеличиваем счетчик
    client_counter += 1

print("Обучение автоэнкодеров завершено!")

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.metrics import MeanSquaredError

# Загружаем данные
df_test_1 = pd.read_csv("modified_data_encode_sorted_datatime.csv")

# Удаляем ненужные столбцы, оставляем transaction_id
df_test_1 = df_test_1.drop(columns=['expiration_date', 'datetime', 'ip'])

# Задаем путь к папке для сохранения моделей
save_dir = 'models'  # Путь к папке, где будут храниться модели

# Функция для загрузки модели для клиента
def load_autoencoder(client_id):
    try:
        # Преобразуем client_id в строку и убираем точку
        client_id_str = str(int(client_id))
        
        # Загружаем модель с явной метрикой
        model = load_model(f'{save_dir}/autoencoder_client_{client_id_str}.keras', custom_objects={'mse': MeanSquaredError()})
        return model
    except Exception as e:
        print(f"Ошибка при загрузке модели для клиента {client_id}: {e}")
        return None

# Масштабируем данные
scaler = StandardScaler()

# Новый столбец для аномальности
anomaly_scores = []

# Счетчик клиентов
client_counter = 1

# Перебираем уникальных клиентов
unique_clients = df_test_1['client_id'].unique()

# Проходим по всем уникальным клиентам
for client_id in unique_clients:
    print(f"Обработка данных для клиента {client_id}... (Клиент {client_counter})")

    # Получаем все транзакции для текущего клиента
    client_data = df_test_1[df_test_1['client_id'] == client_id]

    # Загружаем модель для клиента один раз
    model = load_autoencoder(client_id)
    
    if model:
        # Масштабируем данные для всех транзакций клиента
        client_transactions = client_data.drop(['transaction_id', 'client_id'], axis=1)
        client_transactions_scaled = scaler.fit_transform(client_transactions)

        # Получаем предсказания (реконструкцию) от автоэнкодера
        reconstructed = model.predict(client_transactions_scaled)

        # Вычисляем ошибку реконструкции (MSE) для каждой транзакции
        reconstruction_errors = np.mean(np.square(client_transactions_scaled - reconstructed), axis=1)

        # Определяем аномальности (на основе порога)
        threshold = 0.5  # Порог для определения аномальности (можно настроить)
        client_anomaly_scores = (reconstruction_errors > threshold).astype(int)

        # Добавляем аномальные метки в общий список
        anomaly_scores.extend(client_anomaly_scores)
        
        print(f"Завершена обработка всех транзакций для клиента {client_id}. Клиент {client_counter}.")
    else:
        print(f"Ошибка при загрузке модели для клиента {client_id}.")
        

    # Увеличиваем счетчик клиентов
    client_counter += 1

# Добавляем столбец с результатами в исходный DataFrame
df_test_1['anomaly'] = anomaly_scores

# Сохраняем результат в новый файл (сохранится также и transaction_id)
df_test_1.to_csv("modified_data_with_anomalies2.csv", index=False)

print("Предсказания аномальности добавлены в датасет.")


In [ ]:
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!->>>>

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.metrics import MeanSquaredError

# Загружаем данные
df_test_1 = pd.read_csv("modified_data_encode_sorted_datatime.csv")

# Удаляем ненужные столбцы, оставляем transaction_id
df_test_1 = df_test_1.drop(columns=['expiration_date', 'datetime', 'ip'])

# Задаем путь к папке для сохранения моделей
save_dir = 'models'  # Путь к папке, где будут храниться модели

# Функция для загрузки модели для клиента
def load_autoencoder(client_id):
    try:
        # Преобразуем client_id в строку и убираем точку
        client_id_str = str(int(client_id))
        
        # Загружаем модель с явной метрикой
        model = load_model(f'autoencoder_client_{client_id_str}.h5', custom_objects={'mse': MeanSquaredError()})
        return model
    except Exception as e:
        print(f"Ошибка при загрузке модели для клиента {client_id}: {e}")
        return None

# Масштабируем данные
scaler = StandardScaler()

# Новый столбец для аномальности
anomaly_scores = []

# Счетчик клиентов
client_counter = 1

# Перебираем уникальных клиентов
unique_clients = df_test_1['client_id'].unique()

# Проходим по всем уникальным клиентам
for client_id in unique_clients:
    print(f"Обработка данных для клиента {client_id}... (Клиент {client_counter})")

    # Получаем все транзакции для текущего клиента
    client_data = df_test_1[df_test_1['client_id'] == client_id]

    # Загружаем модель для клиента один раз
    model = load_autoencoder(client_id)
    
    if model:
        # Масштабируем данные для всех транзакций клиента
        client_transactions = client_data.drop(['transaction_id', 'client_id'], axis=1)
        client_transactions_scaled = scaler.fit_transform(client_transactions)

        # Получаем предсказания (реконструкцию) от автоэнкодера
        reconstructed = model.predict(client_transactions_scaled)

        # Вычисляем ошибку реконструкции (MSE) для каждой транзакции
        reconstruction_errors = np.mean(np.square(client_transactions_scaled - reconstructed), axis=1)

        # Определяем аномальности (на основе порога)
        threshold = 0.3  # Порог для определения аномальности (можно настроить)
        client_anomaly_scores = (reconstruction_errors > threshold).astype(bool)

        # Добавляем аномальные метки в общий список
        anomaly_scores.extend(client_anomaly_scores)
        
        print(f"Завершена обработка всех транзакций для клиента {client_id}. Клиент {client_counter}.")
    else:
        print(f"Ошибка при загрузке модели для клиента {client_id}.")
        

    # Увеличиваем счетчик клиентов
    client_counter += 1

# Добавляем столбец с результатами в исходный DataFrame
df_test_1['anomaly'] = anomaly_scores

# Сохраняем результат в новый файл (сохранится также и transaction_id)
df_test_1.to_csv("modified_data_with_anomalies03.csv", index=False)

print("Предсказания аномальности добавлены в датасет.")


# Создаем новый DataFrame только с метками аномальности
anomaly_df = pd.DataFrame(anomaly_scores, columns=['anomaly'])

# Сохраняем новый файл без заголовков и индексов
anomaly_df.to_csv("preds03.csv", header=False, index=False)

print("Результаты аномальности сохранены в новый файл (без заголовков и индексов).")

Обработка данных для клиента 969838140... (Клиент 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


Завершена обработка всех транзакций для клиента 969838140. Клиент 1.
Обработка данных для клиента 198328433... (Клиент 2)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


Завершена обработка всех транзакций для клиента 198328433. Клиент 2.
Обработка данных для клиента 527729205... (Клиент 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


Завершена обработка всех транзакций для клиента 527729205. Клиент 3.
Обработка данных для клиента 298813281... (Клиент 4)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


Завершена обработка всех транзакций для клиента 298813281. Клиент 4.
Обработка данных для клиента 443427305... (Клиент 5)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


...

Завершена обработка всех транзакций для клиента 319941461. Клиент 995.
Обработка данных для клиента 526745565... (Клиент 996)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


Завершена обработка всех транзакций для клиента 526745565. Клиент 996.
Обработка данных для клиента 364757022... (Клиент 997)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


Завершена обработка всех транзакций для клиента 364757022. Клиент 997.
Обработка данных для клиента 828308371... (Клиент 998)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


Завершена обработка всех транзакций для клиента 828308371. Клиент 998.
Обработка данных для клиента 191845802... (Клиент 999)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


Завершена обработка всех транзакций для клиента 191845802. Клиент 999.
Обработка данных для клиента 860457500... (Клиент 1000)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Завершена обработка всех транзакций для клиента 860457500. Клиент 1000.
Предсказания аномальности добавлены в датасет.
Результаты аномальности сохранены в новый файл (без заголовков и индексов).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os

# Загружаем данные
df = pd.read_csv("modified_data_encode_sorted_datatime.csv")



# Удаляем ненужные столбцы (например, идентификаторы или текстовые поля)
df = df.drop(columns=['transaction_id', 'expiration_date', 'ip', 'datetime'])

# Задаем путь к папке для сохранения моделей
save_dir = 'models'  # Путь к папке, где будут храниться модели

# Если папка не существует, создаем её
if not os.path.exists(save_dir):
    os.makedirs(save_dir)



# Получаем список уникальных клиентов
unique_clients = df['client_id'].unique()

# Функция для создания автоэнкодера
def create_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    # Кодировщик
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(32, activation='relu')(encoded)
    encoded = Dense(16, activation='relu')(encoded)
    
    # Декодировщик
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    
    # Создаем модель автоэнкодера
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# Цикл по каждому клиенту
for client_id in unique_clients:
    print(f"Обработка данных для клиента {client_id}...")
    
    # Фильтруем данные для конкретного клиента
    client_data = df[df['client_id'] == client_id]
    
    # Удаляем client_id из данных (он больше не нужен как признак)
    client_data = client_data.drop(columns=['client_id'])
    
    # Масштабируем данные
    scaler = StandardScaler()
    client_data_scaled = scaler.fit_transform(client_data)
    
    # Разделяем данные на обучающую и тестовую выборки
    X_train, X_test = train_test_split(client_data_scaled, test_size=0.2, random_state=42)
    
    # Создаем автоэнкодер
    input_dim = X_train.shape[1]
    autoencoder = create_autoencoder(input_dim)
    
    # Обучаем автоэнкодер
    autoencoder.fit(
        X_train, X_train,
        epochs=50,
        batch_size=32,
        shuffle=True,
        validation_data=(X_test, X_test),
        verbose=1
    )
    
    # Сохраняем модель для клиента
    autoencoder.save(f'{save_dir}/autoencoder_client_{client_id}.keras')
    print(f"Модель для клиента {client_id} сохранена в папку '{save_dir}'.")

print("Обучение автоэнкодеров завершено!")

In [5]:
#здесь код для обучения с помощью многозадачности, а так же гпу 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler
from concurrent.futures import ThreadPoolExecutor
from tensorflow.keras.layers import Dropout
import os


# Проверяем, есть ли доступный GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU доступен")
    # Ограничиваем использование памяти GPU (опционально)
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("GPU не найден")



# Задаем путь к папке для сохранения моделей
save_dir = 'models2'  # Путь к папке, где будут храниться модели

# Если папка не существует, создаем её
if not os.path.exists(save_dir):
    os.makedirs(save_dir)



# Загружаем данные
df = pd.read_csv("modified_data_encode_sorted_datatime.csv")

# Удаляем ненужные столбцы
df = df.drop(columns=['transaction_id', 'expiration_date', 'datetime', 'ip'])



# Получаем список уникальных клиентов
unique_clients = df['client_id'].unique()

# Функция для создания автоэнкодера
def create_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    # Кодировщик
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dropout(0.3)(encoded)  # 30% Dropout
    encoded = Dense(32, activation='relu')(encoded)
    encoded = Dropout(0.3)(encoded)
    encoded = Dense(16, activation='relu')(encoded)
    
    # Декодировщик
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    
    # Создаем модель автоэнкодера
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# Функция для обучения модели для конкретного клиента
def train_autoencoder(client_id):
    print(f"Обработка данных для клиента {client_id}...")
    
    # Фильтруем данные для конкретного клиента
    client_data = df[df['client_id'] == client_id]
    
    # Удаляем client_id из данных
    client_data = client_data.drop(columns=['client_id'])
    
    # Масштабируем данные
    scaler = StandardScaler()
    client_data_scaled = scaler.fit_transform(client_data)
    
    # Создаем автоэнкодер
    input_dim = client_data_scaled.shape[1]
    autoencoder = create_autoencoder(input_dim)
    
    # Обучаем автоэнкодер на всех данных клиента (без разделения на обучающую и тестовую выборки)
    autoencoder.fit(
        client_data_scaled, client_data_scaled,
        epochs=70,
        batch_size=8,
        shuffle=True,
        verbose=1
    )
    
    # Сохраняем модель для клиента
    autoencoder.save(f'{save_dir}/autoencoder_client_{client_id}.keras')
    print(f"Модель для клиента {client_id} сохранена в папку '{save_dir}'.")

# Параллельное обучение моделей для клиентов
with ThreadPoolExecutor(max_workers=1) as executor:  # Установите max_workers в зависимости от вашего процессора/ГПУ
    executor.map(train_autoencoder, unique_clients)

print("Обучение автоэнкодеров завершено!")


GPU не найден
Обработка данных для клиента 969838140...
Epoch 1/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.0921  
Epoch 2/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0444 
Epoch 3/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0773 
Epoch 4/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0966 
Epoch 5/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0937 
Epoch 6/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0465 
Epoch 7/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9777 
Epoch 8/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9418 
Epoch 9/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9550 
Epoch 10/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8441 
Epoch 11/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8686 
Epoch 12/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8303 
Epoch 13/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8060 
Epoch 14/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.8483 
Epoch 15/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0509 
Epoch 4/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0349 
Epoch 5/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9957 
Epoch 6/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9922 
Epoch 7/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8839 
Epoch 8/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8867 
Epoch 9/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8662 
Epoch 10/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8710 
Epoch 11/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8365 
Epoch 12/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8330 
Epoch 13/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8414 
Epoch 14/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8193 
Epoch 15/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8606 
Epoch 16/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8548 
Epoch 17/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8166 
Epoch 18/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7188 
Epoch 31/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6720 
Epoch 32/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6978 
Epoch 33/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6923 
Epoch 34/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6862 
Epoch 35/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7535 
Epoch 36/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6837 
Epoch 37/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6765 
Epoch 38/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6918 
Epoch 39/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6913 
Epoch 40/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7238 
Epoch 41/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7215 
Epoch 42/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7004 
Epoch 43/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6870 
Epoch 44/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6493 
Epoch 45/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 

...

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7485 
Epoch 38/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6896 
Epoch 39/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7229 
Epoch 40/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7326 
Epoch 41/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7151 
Epoch 42/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6794 
Epoch 43/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6931 
Epoch 44/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6802 
Epoch 45/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6928 
Epoch 46/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6838 
Epoch 47/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6544 
Epoch 48/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6813 
Epoch 49/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6277 
Epoch 50/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6787 
Epoch 51/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6679 
Epoch 52/70
7/7 ━━━━━━━━━━━━━━━━━━━━ 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7846 
Epoch 34/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7862 
Epoch 35/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7427 
Epoch 36/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7774 
Epoch 37/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7761 
Epoch 38/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7830 
Epoch 39/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7575 
Epoch 40/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8071 
Epoch 41/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7766 
Epoch 42/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7755 
Epoch 43/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7400 
Epoch 44/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7664 
Epoch 45/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7312 
Epoch 46/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7596 
Epoch 47/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7292 
Epoch 48/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7461 
Epoch 34/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7968 
Epoch 35/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7593 
Epoch 36/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7068 
Epoch 37/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7523 
Epoch 38/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7785 
Epoch 39/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7838 
Epoch 40/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7706 
Epoch 41/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7880 
Epoch 42/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6306 
Epoch 43/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7297 
Epoch 44/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7257 
Epoch 45/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6838 
Epoch 46/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7198 
Epoch 47/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6879 
Epoch 48/70
8/8 ━━━━━━━━━━━━━━━━━━━━ 